In [2]:
from datetime import date
import warnings
warnings.filterwarnings('ignore')

In [3]:
from datetime import date

max_holding = 100
source = "CSV"

#dateTimeFormat = '%Y-%m-%d'
dateTimeFormat = '%d-%b-%Y'

# filter
days = 30 * 24

# frequency
freq = "1D"
symbol = 'BHARTIARTL'
myDate="31_Mar_2019"
filename ="./data/"+symbol+"_"+myDate+".csv"

# Calculate indicators and fields
%run "freedom_indicators_calculation.ipynb"


########## Algorithm #############
from enum import Enum
class S(Enum):
    BUY = 1
    SELL = 2
    HOLD = 3
    
class Trend(Enum):
    SIDE = 0
    UP = 1
    SUP = 2
    DOWN = 3
    SDOWN = 4

In [34]:
from talib import LINEARREG, BETA, LINEARREG_INTERCEPT, LINEARREG_SLOPE, STDDEV, TSF, ADOSC, VAR

# Math
price.priceSD = STDDEV(price.close)
price.priceVAR = VAR(price.close)

price.delBBUP = (price.bbup - price.close)/(price.bbup-price.bbmid)
price.delBBLOW = (price.close - price.bblow)/(price.bbmid-price.bblow)

price.price1dGradient = np.arctan(price.close - price.close.shift(1)) * 180 / np.pi
price.price2dGradient = np.arctan(price.close - price.close.shift(2)) * 180 / np.pi
price.price3dGradient = np.arctan(price.close - price.close.shift(3)) * 180 / np.pi
price.price4dGradient = np.arctan(price.close - price.close.shift(4)) * 180 / np.pi
price.price5dGradient = np.arctan(price.close - price.close.shift(5)) * 180 / np.pi
price.price10dGradient = np.arctan(price.close - price.close.shift(10)) * 180 / np.pi
price.price15dGradient = np.arctan(price.close - price.close.shift(15)) * 180 / np.pi

price.macd1dGradient = price.macd + 1 * (price.macd - price.macd.shift(1))


price.adosc = ADOSC(price.high, price.low, price.close, price.volume)

price.macdLR = LINEARREG(price.macd)
price.macdTSF = TSF(price.macd)

price.priceTSF = TSF(price.close)
price.priceLR = LINEARREG(price.close)


price.trend1D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
price.trend1D[price1dGradient >= 45] = Trend.UP
price.trend1D[price1dGradient <= -45] = Trend.DOWN
price.trend1D[price1dGradient >= 65] = Trend.SUP
price.trend1D[price1dGradient <= -65] = Trend.SDOWN

price.trend3D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
price.trend3D[price3dGradient >= 45] = Trend.UP
price.trend3D[price3dGradient <= -45] = Trend.DOWN
price.trend3D[price3dGradient >= 65] = Trend.SUP
price.trend3D[price3dGradient <= -65] = Trend.SDOWN

price.trend5D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
price.trend5D[price5dGradient >= 45] = Trend.UP
price.trend5D[price5dGradient <= -45] = Trend.DOWN
price.trend5D[price5dGradient >= 65] = Trend.SUP
price.trend5D[price5dGradient <= -65] = Trend.SDOWN

price.trend10D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
price.trend10D[price10dGradient >= 45] = Trend.UP
price.trend10D[price10dGradient <= -45] = Trend.DOWN
price.trend10D[price10dGradient >= 65] = Trend.SUP
price.trend10D[price10dGradient <= -65] = Trend.SDOWN

price.trend15D = pd.DataFrame(data = {"Trend":np.full(price.index.shape[0], Trend.SIDE)},index=price.index)
price.trend15D[price15dGradient >= 45] = Trend.UP
price.trend15D[price15dGradient <= -45] = Trend.DOWN
price.trend15D[price15dGradient >= 65] = Trend.SUP
price.trend15D[price15dGradient <= -65] = Trend.SDOWN

price.chandlierPoint = price.close-price.atr

price.sdHighLow = STDDEV((price.high - price.close))
price.varHighLow = STDDEV((price.high - price.close))
price.delPriceHighLow = (price.high - price.close)/sdHighLow


# Heikin-Ashi
price.priceHAOpen = (price.open.shift(1) + price.close.shift(1))/2
price.priceHAHigh = price.max(axis = 1, skipna = True)
price.priceHALow = price.min(axis = 1, skipna = True)
price.priceHAClose = (price.open+price.high+price.low+price.close)/4

In [31]:
price.macd1dGradient[1:60]

2017-06-01     4.195296
2017-06-02     5.840476
2017-06-05     5.271994
2017-06-06     3.885040
2017-06-07     4.780333
2017-06-08     2.890544
2017-06-09     2.629421
2017-06-12     1.541311
2017-06-13     0.655284
2017-06-14     1.166172
2017-06-15     0.938905
2017-06-16     0.654421
2017-06-19     1.377353
2017-06-20     1.006689
2017-06-21     1.143557
2017-06-22     0.719239
2017-06-23     0.641847
2017-06-27     2.249112
2017-06-28     4.061082
2017-06-29     5.733582
2017-06-30     4.844702
2017-07-03     5.501053
2017-07-04     4.512711
2017-07-05     3.387798
2017-07-06     4.781238
2017-07-07     6.089703
2017-07-10    12.262549
2017-07-11     8.949964
2017-07-12     9.232127
2017-07-13    10.883598
2017-07-14    11.834082
2017-07-17    11.985213
2017-07-18    11.572165
2017-07-19    15.284767
2017-07-20    14.457491
2017-07-21    11.279341
2017-07-24    13.487009
2017-07-25    15.441882
2017-07-26    14.448215
2017-07-27    10.621287
2017-07-28     9.065475
2017-07-31    10

In [35]:
#price.dropna(inplace=True)
labels = pd.DataFrame(data = {"holdings":np.zeros(price.index.shape[0])},index=price.index)
#Entry Criterion
labels[
    (macd1dGradient > price.macdsignal)
] = S.BUY

#Exit Criterion
'''labels[
     
] = S.SELL
'''

# buy and sell decisions are stored in arrays for plotting purpose(visualisation)

buy = price[labels['holdings']==S.BUY]
sell = price[labels['holdings']==S.SELL]

In [36]:
#################################
#   Plotting Data              #
###############################

rangeMin = price.index[-150]
rangeMax = price.index[-1]

%run "freedom_plot.ipynb"

This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x1,y2 ]
[ (3,1) x1,y3 ]
[ (4,1) x1,y4 ]
[ (5,1) x1,y5 ]
[ (6,1) x1,y6 ]
[ (7,1) x1,y7 ]
[ (8,1) x1,y8 ]



## Entry Criterion
- MACD crossover when market is trending up
- BB when market is moving sideways

## Exit Criterion
- Stochastics

## Identify market trend
- low ATR means market moving sideways
- Flat(gradient=0) MACD means market is moving sideways
- low ATR indicates low volatility - market is moving sideways
- High ATR indicates high volatility - market is trending
- Gradient of current - last 15 days